In [1]:
import warnings

# Filter out specific warning messages
warnings.filterwarnings("ignore", category=UserWarning, module="sdRDM")

In [2]:
%reload_ext autoreload
%autoreload 2
from EnzymePynetics import Estimator

In [3]:
estimator, enzymeml = Estimator.from_enzymeml(
    enzymeml="simulated_enzymeML.json",
    measured_reactant="substrate",
)

In [4]:
substrate, product = estimator.reactants
enzyme = estimator.enzymes[0]

In [5]:
# Define Reaction
oxidation = estimator.add_reaction(
    id="r1", name="Oxidation", educt=substrate, product=product, enzyme=enzyme
)

In [6]:
# Define potential substrate rate-laws
michaelis = estimator.add_model(
    id="model1",
    name="michaelis-menten",
    equation="substrate = -substrate * enzyme * k_cat / (K_m + substrate)",
)

competitive_product = estimator.add_model(
    id="model3",
    name="competitive product inhibition",
    equation="substrate = -substrate * enzyme * k_cat / (K_m * (1 + product / K_ic) + substrate)",
)

substrate_inhibition = estimator.add_model(
    id="model4",
    name="substrate inhibition",
    equation="substrate = -k_cat * enzyme * substrate / (K_m + ((1+(substrate/K_iu))*substrate))",
)

# Define potential enzyme rate-laws
enzyme_inactivation = estimator.add_model(
    id="model2",
    name="enzyme inactivation",
    equation="enzyme = -k_ie * enzyme",
)

In [7]:
estimator.fit_models()

Fitting michaelis-menten
Fitting michaelis-menten with enzyme inactivation
Fitting competitive product inhibition
Fitting competitive product inhibition with enzyme inactivation
Fitting substrate inhibition
Fitting substrate inhibition with enzyme inactivation


,AIC,k_cat,K_m,K_ic,K_iu,k_ie
,,1 / min,mmol / l,mmol / l,mmol / l,1 / min
Model,,,,,,
competitive product inhibition with enzyme inactivation,-3140,12.00,17.00,10.00,,0.10
michaelis-menten with enzyme inactivation,-133,11.60,21.06,,,0.20
substrate inhibition with enzyme inactivation,-127,11.85,21.75,,3000.00,0.20
competitive product inhibition,-121,14.03,11.95,2.22,,
michaelis-menten,117,7.81,33.54,,,
substrate inhibition,119,7.99,34.60,,3000.00,


In [19]:
estimator.visualize()

In [11]:
# Get the best model system
model_system = estimator.get_reaction_system(
    "competitive product inhibition with enzyme inactivation")

# Safe the model system to the EnzymeML document
enzymeml_with_kinetic_model = estimator.to_enzymeml(enzymeml, model_system)

In [14]:
for reaction in enzymeml_with_kinetic_model.reactions:
    print(reaction)

Reaction
├── id = r1
├── name = Oxidation
├── reversible = False
├── temperature = 30.0
├── temperature_unit = C
├── ph = 4.0
├── ontology = SBO:0000176
├── model
│   └── KineticModel
│       ├── id = model3
│       ├── name = competitive product inhibition
│       ├── equation = substrate = -substrate * enzyme * k_cat / (K_m * (1 + product / K_ic) + substrate)
│       └── parameters
│           ├── 0
│           │   └── KineticParameter
│           │       ├── id = kineticparameter2
│           │       ├── name = k_cat
│           │       ├── value = 11.999999999999083
│           │       ├── unit = 1 / min
│           │       ├── initial_value = 8.582935014291738
│           │       ├── upper = 8582.93501429174
│           │       ├── lower = 0.008582935014291738
│           │       ├── is_global = False
│           │       ├── stdev = 3.820108731971233e-12
│           │       ├── constant = False
│           │       └── ontology = SBO:0000025
│           ├── 1
│           │   └── Ki